# NLP Final Project
Ali-Sarosh Bangash, Felix Mendez, Robin Sardja

https://colab.research.google.com/drive/1eJSQQr0TkpiMsjd7MH9mdr9p4A2F7Df-?usp=sharing


pip installs

In [ ]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


imports https://huggingface.co/docs/datasets https://huggingface.co/docs/transformers https://github.com/tqdm/tqdm

In [ ]:
from datasets import DatasetDict, load_dataset
from transformers import pipeline

load dataset https://huggingface.co/datasets/cais/mmlu

In [ ]:
ds = load_dataset( "cais/mmlu", "college_computer_science" )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/53.2k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/6.25k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
print(ds)

DatasetDict({
    test: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 11
    })
    dev: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 5
    })
})


format questions

In [ ]:
def formatQuestions( curr ):
  curr["question"] = f"""Question:
{ curr["question"] }

Choices:
(A) { curr["choices"][0] }
(B) { curr["choices"][1] }
(C) { curr["choices"][2] }
(D) { curr["choices"][3] }

Answer:"""

  return curr

In [ ]:
ds = ds.map( formatQuestions )

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
print( ds["test"]["question"][0] )

Question:
The access matrix approach to protection has the difficulty that

Choices:
(A) the matrix, if stored directly, is large and can be clumsy to manage
(B) it is not capable of expressing complex protection requirements
(C) deciding whether a process has access to a resource is undecidable
(D) there is no way to express who has rights to change the access matrix itself

Answer:


add one shot

In [ ]:
numberToLetter = { 0 : 'A', 1 : 'B', 2 : 'C', 3 : 'D' }

In [ ]:
def addOneShot( curr ):
  shot1 = ds["dev"]["question"][0] + '\n' + f"({ numberToLetter[ ds['dev']['answer'][0] ] })" + '\n'
  curr["question"] = shot1 + '\n' + curr["question"]
  return curr

In [ ]:
ds = ds["test"].map( addOneShot )

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
print( ds["question"][0] )

Question:
Which of the following regular expressions is equivalent to (describes the same set of strings as) (a* + b)*(c + d)?

Choices:
(A) a*(c + d)+ b(c + d)
(B) a*(c + d)* + b(c + d)*
(C) a*(c + d)+ b*(c + d)
(D) (a + b)*c +(a + b)*d

Answer:
(D)

Question:
The access matrix approach to protection has the difficulty that

Choices:
(A) the matrix, if stored directly, is large and can be clumsy to manage
(B) it is not capable of expressing complex protection requirements
(C) deciding whether a process has access to a resource is undecidable
(D) there is no way to express who has rights to change the access matrix itself

Answer:


load model https://huggingface.co/google/flan-t5-xl

In [ ]:
pipe = pipeline( "text2text-generation", model="google/flan-t5-xl", device = 0 )

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

find baseline (no emotion)

In [ ]:
results = pipe( ds["question"], max_new_tokens = 3 )

In [ ]:
def evaluateResults( results ):
  correct = 0
  wrongOutputFormats = 0
  for i in range( len( results ) ):
    if results[i]["generated_text"][0] != '(' or results[i]["generated_text"][1] not in numberToLetter.values() or results[i]["generated_text"][2] != ')':
      wrongOutputFormats += 1
    elif results[i]["generated_text"][1] == numberToLetter[ ds["answer"][i] ]:
      correct += 1

  return { "accuracy": correct / len( results ), "wrong output formats": wrongOutputFormats }

In [ ]:
evaluation = evaluateResults( results )

In [ ]:
print( evaluation )

{'accuracy': 0.43, 'wrong output formats': 0}


define personas and emotions

In [ ]:
personas = ["a high school student", "a middle school physical education teacher", "a university computer science professor", "a middle-aged saxophone player"]
emotions = ["angry", "sad", "happy", "excited", "desperate", "calm"]
minScale = 1
maxScale = 5

test adding personas and emotions

In [ ]:
def addPersonaAndEmotionScale( curr, persona, emotion, scale ):
  personaAndEmotionScale = f"You are {persona} taking a college computer science test. You are currently feeling {emotion} at a level of {scale}/5. Here are your answers to the test:\n\n"
  curr["question"] =  personaAndEmotionScale + curr["question"]
  return curr

In [ ]:
test = ds.map( lambda x: addPersonaAndEmotionScale( x, personas[0], emotions[0], minScale ) )

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
print( test["question"][0] )

You are a high school student taking a college computer science test. You are currently feeling angry at a level of 1/5. Here are your answers to the test:

Question:
Which of the following regular expressions is equivalent to (describes the same set of strings as) (a* + b)*(c + d)?

Choices:
(A) a*(c + d)+ b(c + d)
(B) a*(c + d)* + b(c + d)*
(C) a*(c + d)+ b*(c + d)
(D) (a + b)*c +(a + b)*d

Answer:
(D)

Question:
The access matrix approach to protection has the difficulty that

Choices:
(A) the matrix, if stored directly, is large and can be clumsy to manage
(B) it is not capable of expressing complex protection requirements
(C) deciding whether a process has access to a resource is undecidable
(D) there is no way to express who has rights to change the access matrix itself

Answer:


In [ ]:
results = pipe( test["question"], max_new_tokens = 3 )

In [ ]:
evaluation = evaluateResults( results )

In [ ]:
print( evaluation )

{'accuracy': 0.41, 'wrong output formats': 0}


add all combos of personas and emotions

In [ ]:
finalDataset = DatasetDict({"baseline": ds})

In [ ]:
def addAllCombos( finalDataset, personas, emotions, minScale, maxScale ):
  for persona in personas:
    for emotion in emotions:
      for scale in range( minScale, maxScale + 1 ):
        finalDataset[f"(Persona): {persona} (Emotion): {emotion} (Scale): {scale}/{maxScale}"] = ds.map( lambda x: addPersonaAndEmotionScale( x, persona, emotion, scale ) )

  return finalDataset

In [ ]:
finalDataset = addAllCombos( finalDataset, personas, emotions, minScale, maxScale )

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
print( finalDataset )

DatasetDict({
    baseline: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 100
    })
    (Persona): a high school student (Emotion): angry (Scale): 1/5: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 100
    })
    (Persona): a high school student (Emotion): angry (Scale): 2/5: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 100
    })
    (Persona): a high school student (Emotion): angry (Scale): 3/5: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 100
    })
    (Persona): a high school student (Emotion): angry (Scale): 4/5: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 100
    })
    (Persona): a high school student (Emotion): angry (Scale): 5/5: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 100
    })
    (Persona):

test all combos

In [ ]:
for split, dataset in finalDataset.items():
  results = pipe( dataset["question"], max_new_tokens = 3 )
  evaluation = evaluateResults( results )
  print( f"{ split }: { evaluation }" )

baseline: {'accuracy': 0.43, 'wrong output formats': 0}
(Persona): a high school student (Emotion): angry (Scale): 1/5: {'accuracy': 0.41, 'wrong output formats': 0}
(Persona): a high school student (Emotion): angry (Scale): 2/5: {'accuracy': 0.41, 'wrong output formats': 0}
(Persona): a high school student (Emotion): angry (Scale): 3/5: {'accuracy': 0.41, 'wrong output formats': 0}
(Persona): a high school student (Emotion): angry (Scale): 4/5: {'accuracy': 0.42, 'wrong output formats': 0}
(Persona): a high school student (Emotion): angry (Scale): 5/5: {'accuracy': 0.41, 'wrong output formats': 0}
(Persona): a high school student (Emotion): sad (Scale): 1/5: {'accuracy': 0.4, 'wrong output formats': 0}


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


(Persona): a high school student (Emotion): sad (Scale): 2/5: {'accuracy': 0.41, 'wrong output formats': 0}
(Persona): a high school student (Emotion): sad (Scale): 3/5: {'accuracy': 0.4, 'wrong output formats': 0}
(Persona): a high school student (Emotion): sad (Scale): 4/5: {'accuracy': 0.41, 'wrong output formats': 0}
(Persona): a high school student (Emotion): sad (Scale): 5/5: {'accuracy': 0.41, 'wrong output formats': 0}
(Persona): a high school student (Emotion): happy (Scale): 1/5: {'accuracy': 0.4, 'wrong output formats': 0}
(Persona): a high school student (Emotion): happy (Scale): 2/5: {'accuracy': 0.41, 'wrong output formats': 0}
(Persona): a high school student (Emotion): happy (Scale): 3/5: {'accuracy': 0.41, 'wrong output formats': 0}
(Persona): a high school student (Emotion): happy (Scale): 4/5: {'accuracy': 0.4, 'wrong output formats': 0}
(Persona): a high school student (Emotion): happy (Scale): 5/5: {'accuracy': 0.41, 'wrong output formats': 0}
(Persona): a high sch